In [24]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

import shutil
from tqdm import tqdm_notebook

device = "cuda" if torch.cuda.is_available() else "cpu"
#Need only 1 GPU if loading 8-bit model
print(device)

print("Using %d GPUs" %torch.cuda.device_count())

import gradio as gr
import time


cuda
Using 4 GPUs


In [21]:
chunk_size = 1024
chunk_overlap = 0
seq_length = 1024
embedding_model_name = "all-mpnet-base-v2"

In [22]:
model_name = "eachadea/vicuna-13b-1.1"
tokenizer_path = "./tokenizer/"

#Create a local tokenizer copy the first time
if os.path.isdir(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained("model_name")
    os.mkdir(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")#, load_in_8bit=True)
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=seq_length,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
#Load embedding model and use that to embed text from source
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

init_docs = True
use_openai = False

if use_openai:
    embeddings = OpenAIEmbeddings(openai_api_key='')
    embed_path = 'embeds/openai'
else:
    embed_path = 'embeds/%s' %(embedding_model_name)


if init_docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    if os.path.exists(embed_path):
        response = input("WARNING: DELETING EXISTING EMBEDDINGS. Type \"y\" to continue.")
        if response.strip() == "y":
            shutil.rmtree(embed_path)
        else:
            raise ValueError("Existing Chroma Collection")

    all_texts = []
    doc_path = 'APS-Science-Highlight'
    for text_fp in os.listdir(doc_path):
        with open(os.path.join(doc_path, text_fp), 'r') as text_f:
            book = text_f.read()
        texts = text_splitter.split_text(book)
        all_texts += texts

    docsearch = Chroma.from_texts(
        all_texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(all_texts))],
        persist_directory=embed_path
    )
    docsearch.persist()
else:
    docsearch = Chroma(persist_directory=embed_path)

/tmp/ipykernel_627644/275823926.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  all_texts, embeddings, metadatas=[{"source": str(i)} for i in tqdm_notebook(range(len(all_texts)))],


  0%|          | 0/9133 [00:00<?, ?it/s]

Using embedded DuckDB with persistence: data will be stored in: embeds/all-mpnet-base-v2


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [6]:
def get_context(query):
    N_hits = 3 #How many hits of context to provide?
    docs = docsearch.similarity_search_with_score(query, k=N_hits)
    #Get context strings
    context=""
    for i in range(N_hits):
        context += docs[i][0].page_content +"\n"
    print (context)
    return context

In [37]:
query1 = "Who is Jonathan Lang"
query2 = "What is CDI"
query3 = "What is 3DCDI-NN?"
get_context(query1)
N_hits = 5
docs = docsearch.similarity_search_with_score(query1, k=N_hits)
for doc in docs:
    print(doc[0])

See also: "How LSD Binds to the Brain, and Why Trips Last So Long," by Douglas Main, Newsweek.com, 1.26.17.

Argonne National Laboratory seeks solutions to pressing national problems in science and technology. The nation's first national laboratory, Argonne conducts leading-edge basic and applied scientific research in virtually every scientific discipline. Argonne researchers work closely with researchers from hundreds of companies, universities, and federal, state and municipal agencies to help them solve their specific problems, advance America's scientific leadership and prepare the nation for a better future. With employees from more than 60 nations, Argonne is managed by UChicago Argonne, LLC for the U.S. Department of Energy's Office of Science.
See also: “Fighting Parasitic Infections: Promise in Cyclic Peptides,” NIH Director’s Blog, by Dr. Francis Collins, April 11, 2017.

Argonne National Laboratory seeks solutions to pressing national problems in science and technology. The